In [5]:
import mysql.connector
mydb_conn = mysql.connector.connect(
    user='root',
    password='Passant@01005500317//',
    port = '3306',
    host='127.0.0.1',
    database='library')

print (mydb_conn)

In [6]:
cursor = mydb_conn.cursor()
cursor

In [23]:
## - 1- find the number of availalbe copies of the book (Dracula)

cursor.execute("""
SELECT count( DISTINCT l.bookid ) as Availablecopies FROM loans AS l
INNER JOIN books AS b ON l.bookid = b.bookid  
WHERE b.title = "Dracula"
AND l.bookid NOT IN (
    SELECT DISTINCT bookid 
    FROM loans 
    WHERE returneddate IS NULL)
    """)
result = cursor.fetchall()
result

[(2,)]

In [24]:

## -- 2- check total copies of the book */
cursor.execute("""
 Select * from books
WHERE title = 'Dracula'
    """)
result = cursor.fetchall()
result

[(12, 'Dracula', 'Bram Stoker', 1897, 8232533734),
 (60, 'Dracula', 'Bram Stoker', 1897, 4274716715),
 (73, 'Dracula', 'Bram Stoker', 1897, 8730298424)]

In [25]:
## -- 3- current total loans of the book */

cursor.execute("""
select Count(loanID) as Current_total_Loan from books as b
inner join loans as l on b.bookid = l.bookID
where b.title = "Dracula" 
and l.returneddate is null

    """)
result = cursor.fetchall()
result

[(1,)]

In [15]:
## -- 4- total available books of dracula */

cursor.execute("""
SELECT DISTINCT l.bookid  as totalAvailablecopies FROM loans AS l
INNER JOIN books AS b ON l.bookid = b.bookid  
WHERE b.title = "Dracula"
AND l.bookid NOT IN (
    SELECT DISTINCT bookid 
    FROM loans 
    WHERE returneddate IS NULL
)

    """)
result = cursor.fetchall()
result

[(60,), (12,)]

In [18]:
##-- 5- Add new books to the library  

cursor.execute("""
INSERT INTO Books (BookID,Title,Author,Published,Barcode) VALUES(210, "Microfinanace" , "victor Hugo" , 2000 , 2722805558),
(230 , "Economics" , "Jules Verne" , 1994 , 27112386879)

    """)
result = cursor.fetchall()
result

[]

In [19]:
## -- 6- Check out Books: books(4043822646, 2855934983) whose patron_email(jvaan@wisdompets.com), loandate=2020-08-25, duedate=2020-09-08, loanid=by_your_choice*/

cursor.execute("""
SELECT 
    bo.bookid, 
    bo.title, 
    bo.barcode, 
    lo.loanid, 
    lo.loandate, 
    lo.duedate, 
    lo.returneddate, 
    p.patronID, 
    p.firstname, 
    p.lastname, 
    p.email
FROM books AS bo 
INNER JOIN loans AS lo ON bo.bookid = lo.bookid
INNER JOIN patrons AS p ON lo.patronID = p.patronID
WHERE bo.barcode IN (4043822646, 2855934983)
AND p.email = "jvaan@wisdompets.com"
AND DATE(lo.loandate) = '2020-08-25' 
AND DATE(lo.duedate) = '2020-09-08'
    """)
result = cursor.fetchall()
result

[]

In [ ]:
## -- 7- Check books for Due back    

cursor.execute("""


    """)
result = cursor.fetchall()
result

In [20]:

##-- 8 Return books to the library (which have barcode=6435968624) and return this book at this date(2020-07-05)

cursor.execute("""
UPDATE loans 
SET ReturnedDate = '2020-07-01'
WHERE bookid = (SELECT bookid FROM books WHERE barcode = 6435968624);

    """)
result = cursor.fetchall()
result

[]

In [21]:

##-- 9-  Encourage Patrons to check out books ( generate a report of showing 10 patrons who have checked out the fewest books. )   

cursor.execute("""
SELECT 
    p.firstname, 
    p.lastname, 
    COUNT(*) AS num_books 
FROM patrons AS p 
INNER JOIN loans AS l ON p.patronid = l.patronid
GROUP BY p.patronid, p.firstname, p.lastname  
ORDER BY num_books asc 
LIMIT 10;

    """)
result = cursor.fetchall()
result

[('Caril', 'Matejic', 9),
 ('Alaster', 'Ruggles', 9),
 ('Merle', 'Sukbhans', 11),
 ('Norby', 'Sleight', 12),
 ('Curr', 'Fittall', 12),
 ('Jack', 'Vaan', 13),
 ('Cammi', 'Kynett', 14),
 ('Orton', 'Stavers', 14),
 ('Carla', 'Swinfen', 14),
 ('Aleen', 'Fasey', 14)]

In [22]:
##--  10 Find books to feature for an event (create a list of books from 1890s that are currently available)

cursor.execute("""
select  distinct b.bookID , b.title , b.author from books as b 
inner join loans as l on b.bookid = l.bookid 
where b.published < 1890
AND l.returneddate IS not NULL
order by b.bookID asc
    """)
result = cursor.fetchall()
result

[(1, 'Walden, and On The Duty Of Civil Disobedience', 'Henry David Thoreau'),
 (2, 'The Scarlet Letter', 'Nathaniel Hawthorne'),
 (4, 'The Adventures of Tom Sawyer', 'Mark Twain'),
 (5, 'David Copperfield', 'Charles Dickens'),
 (6, 'A Modest Proposal', 'Jonathan Swift'),
 (9, "Uncle Tom's Cabin", 'Harriet Beecher Stowe'),
 (10, 'The Count of Monte Cristo', 'Alexandre Dumas'),
 (13, 'Oliver Twist', 'Charles Dickens'),
 (20, 'Treasure Island', 'Robert Louis Stevenson'),
 (21, 'The Prince', 'Niccolò Machiavelli'),
 (22, 'Adventures of Huckleberry Finn', 'Mark Twain'),
 (23, 'Great Expectations', 'Charles Dickens'),
 (24, 'Candide', 'Voltaire'),
 (25, 'The Life and Adventures of Robinson Crusoe', 'Daniel Defoe'),
 (27, 'Wuthering Heights', 'Emily Brontë'),
 (29, 'A Modest Proposal', 'Jonathan Swift'),
 (30, 'On the origin of species', 'Charles Darwin'),
 (31,
  "Gulliver's Travels into Several Remote Nations of the World",
  'Jonathan Swift'),
 (35, 'The Brothers Karamazov', 'Fyodor Dostoy

In [26]:
## -- 11 /* Book Statistics (create a report to show how many books were published each year)
cursor.execute("""
SELECT Published, COUNT(DISTINCT(bookid)) AS TotalNumberOfPublishedBooks FROM Books
GROUP BY Published
ORDER BY TotalNumberOfPublishedBooks DESC
""")
result = cursor.fetchall()
result

[(1895, 9),
 (1890, 9),
 (1847, 6),
 (1850, 6),
 (1892, 6),
 (1859, 6),
 (1884, 6),
 (1869, 6),
 (1898, 6),
 (1911, 6),
 (1899, 5),
 (1922, 5),
 (1791, 3),
 (1811, 3),
 (1813, 3),
 (1815, 3),
 (1818, 3),
 (1839, 3),
 (1849, 3),
 (1851, 3),
 (1852, 3),
 (1854, 3),
 (1858, 3),
 (1861, 3),
 (1865, 3),
 (1876, 3),
 (1878, 3),
 (1879, 3),
 (1880, 3),
 (1883, 3),
 (1886, 3),
 (1887, 3),
 (1889, 3),
 (1894, 3),
 (1896, 3),
 (1897, 3),
 (1532, 3),
 (1726, 3),
 (1900, 3),
 (1901, 3),
 (1905, 3),
 (1908, 3),
 (1729, 3),
 (1914, 3),
 (1915, 3),
 (1920, 3),
 (1759, 3),
 (1923, 3),
 (1689, 2),
 (1913, 2),
 (1670, 2),
 (1902, 2),
 (1375, 2),
 (1921, 2),
 (1906, 2),
 (1605, 2),
 (1994, 2),
 (2000, 2)]

In [27]:
## -- 12 Book Statistics (create a report to show 5 most popular Books to check out )
cursor.execute("""
SELECT b.Title, b.Author, b.Published, COUNT(b.Title) AS TotalTimesOfLoans FROM Books b
inner JOIN Loans l ON b.BookID = l.BookID
GROUP BY b.Title, b.Author, b.Published 
ORDER BY TotalTimesOfLoans DESC
LIMIT 5
""")
result = cursor.fetchall()
result

[('Frankenstein; Or, The Modern Prometheus',
  'Mary Wollstonecraft Shelley',
  1818,
  53),
 ('The War of the Worlds', 'H. G. Wells', 1898, 43),
 ('The Awakening, and Selected Short Stories', 'Kate Chopin', 1899, 43),
 ('The Scarlet Letter', 'Nathaniel Hawthorne', 1850, 42),
 ('The Picture of Dorian Gray', 'Oscar Wilde', 1890, 39)]